### 3. This question relates to the task of Digital Signature Verification using RSA. Please,follow the underlying steps:


a. Generate RSA key pairs, where the public key encryption exponent “e” should be 3. And consider the message “m” to be your roll number (all small letters). For eg- "p21cs011".


In [1]:
from sympy import isprime
import random
def randomPrimeGen(n):
    p=""
    for i in range(3*n*n):
        p="1"
        # random string of 1s and 0s of length n-1
        p+=format(random.getrandbits(n-1),'0b')
        if(isprime(int(p,2))):
            return p
    return False
def GenModulus(n):
    p=randomPrimeGen(n)
    q=randomPrimeGen(n)
    while(p==q):
        q=randomPrimeGen(n)
    return int(p,2)*int(q,2),int(p,2),int(q,2)

# N,p,q=GenModulus(512)

# print("The output in base10 is:")
# print("N=",N)
# print("p=",p)
# print("q=",q)

In [5]:
from sympy import gcd
import random
def GenRSA(n):
    N,p,q=GenModulus(n)
    phi=(p-1)*(q-1)
    # Choose an integer e such that 1 < e < phi and gcd(e, phi) = 1
    # e = random.randrange(1, phi)
    e=3 #GIVEN
    g = gcd(e, phi)
    while g != 1:
        # e = random.randrange(1, phi)
        g = gcd(e, phi)
        N,p,q=GenModulus(n)
        phi=(p-1)*(q-1)
    
    # d ≡ e^(-1) (mod phi) [d is the modular multiplicative inverse of e modulo phi]
    d = pow(e, -1, phi)
    print("phi=",phi)
    return N,e,d

N,e,d=GenRSA(128)
print("The output in base10 is:")
print("N=",N)
print("e=",e)
print("d=",d)

        

phi= 628592134224257406714487094916283168090296070355296518291497295571355743096
The output in base10 is:
N= 628592134224257406714487094916283168237507598651646301724670326291196763219
e= 3
d= 419061422816171604476324729944188778726864046903531012194331530380903828731


b. Digitally sign the message “m” using the private key with PKCS1.5 padding to
obtain signed text “x” (Inbuilt libraries support automatic message conversion
followed by encryption). You can use any cryptographic hashing function for the
same.
Note - Private key can be used only once for signing the original message.


In [6]:
import hashlib
import os

def pkcs1_v1_5_pad(message, key_size):
    # Calculate the hash of the message
    hash = hashlib.sha256(message).digest()

    # Create the ASN.1 prefix for SHA-256
    prefix = bytes.fromhex('3031300d060960864801650304020105000420')

    # Calculate the number of padding bytes needed
    padding_length = key_size - len(prefix) - len(hash) - 3

    # Create the padding
    padding = b'\xff' * padding_length

    # Concatenate everything together to form the padded message
    padded_message = b'\x00\x02' + padding + b'\x00' + prefix + hash

    return padded_message

pad_M = pkcs1_v1_5_pad(b'Hello, world!', 512)
print("Padded message is:")
print(pad_M)
# convert to integer
M = int.from_bytes(pad_M, byteorder='big')
print("Padded message in integer is:")
print(M)

# convert m to bytes
m = M.to_bytes((M.bit_length() + 7) // 8, byteorder='big')
print("Padded message in bytes is:")
print(m)

Padded message is:
b'\x00\x02\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xf

In [ ]:
def encrypt(plainText,publicKey):
    plainText=int.from_bytes(plainText, byteorder='big')
    N,e=publicKey
    return pow(plainText,e,N)
def decrypt(cipherText,privateKey):
    N,d=privateKey
    plainText=pow(cipherText,d,N)
    plainText=plainText.to_bytes((plainText.bit_length() + 7) // 8, byteorder='big')
    plainText=plainText.decode('utf-8')
    return plainText


In [ ]:
def pkcs1_v1_5_verify(padded_message, key_size):
    # Check the initial bytes
    if padded_message[:2] != b'\x00\x01':
        return False

    # Find the position of the 00h byte that separates the padding from the prefix and hash
    separator_index = padded_message.find(b'\x00', 2)

    # Check if the separator was found and if it's in the correct position
    if separator_index < 0 or separator_index > key_size - len(hashlib.sha256().digest()) - len(bytes.fromhex('3031300d060960864801650304020105000420')) - 1:
        return False

    # Check the padding
    padding = padded_message[2:separator_index]
    if padding != b'\xff' * len(padding):
        return False

    # If all checks passed, the padding is valid
    return True

# Test the function
print(pkcs1_v1_5_verify(pad_M, 512))  # Should print: True

True


c. Design a server (or function) that accepts the ciphertext, authenticates the sender
by decrypting the signed message using the public key and verifying the message
(plaintext) as per preliminaries - “Message Verification”. The server returns a
boolean value indicating the verification results. For instance - If the format is not
valid, the server responds with “False” value. On the other hand, if the format is
valid, the server returns “True”.
That is, there exists a PKCS Padding oracle to tell you if the padding of a message is valid or not.


In [ ]:
def padding_format_oracle(signature, public_key):
    try:
        # Verify the signature with the public key
        public_key.verify(
            signature,
            message.encode(),
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        return True
    except:
        return False

Your task is to generate another ciphertext that passes the PKCS padding oracle
verification. Importantly, the direct use of both public & private keys is prohibited. The
task should be accomplished using only the PKCS padding oracle